# Psychoinformatics - Week 14 (Exercises)
by your name (your email)

## 1 教學意見調查 (4 points)
Please provide a screenshot showing that you've completed the evaluation of this course, and thanks for your feedback!

In [ ]:
# Please write your codes here

## 2 Audio Segementation (4 points) 
Please use one audio/speech segementation method of your choice (including your own codes) to calculate:

[1] the number of segments, and <br>
[2] the total speech duration of all the segments (seconds)

of each .wav file in https://ceiba.ntu.edu.tw/course/4671ea/content/speech.zip .

In the following example, there are 5 segements and the total speech duration is the sum of yellow(+blue) periods.

<img src="https://www.irit.fr/recherches/SAMOVA/assets/images/segmentation3.gif">

In [ ]:
# Please write your codes here